## Postprocessing
> Postprocessing step to cleanup elimination events.

In [16]:
#| default_exp segmentation.postprocessing
#| export
import dataclasses
import logging

from csgo_clips_autotrim.experiment_utils.utils import getLogger
from csgo_clips_autotrim.segmentation.elimination import EliminationSegmentationResult, is_bbox_overlap, bbox_area

logger = getLogger('postprocessing')
logger.setLevel(logging.DEBUG)

In [55]:
#| export
def remove_duplicate_events(result: EliminationSegmentationResult) -> EliminationSegmentationResult:
    """Remove duplicate elimination events (overlapping event bboxes) over the
    same region.

    Args:
        result (EliminationSegmentationResult): input result

    Returns:
        EliminationSegmentationResult
    """
    deduped_events = []
    marked = [False for _ in range(len(result.elimination_events))]

    for i, event1 in enumerate(result.elimination_events):
        if marked[i]:
            continue
        
        biggest = event1
        logger.debug('Got event %d: %s' % (i, event1))

        for j, event2 in enumerate(result.elimination_events[i+1:]):
            if is_bbox_overlap(biggest.event.bbox, event2.event.bbox):
                marked[i + j + 1] = True
                logger.debug('Bbox overlap: %s' % (event2,))
                if bbox_area(biggest.event.bbox) < bbox_area(event2.event.bbox):
                    biggest = event2
                
        logger.debug(marked)
        if biggest == event1:
            logger.debug('Adding event')
            deduped_events.append(biggest)
    
    return dataclasses.replace(result, elimination_events=deduped_events)

### Test run

In [56]:
from csgo_clips_autotrim.experiment_utils.constants import BASE_DIR


sample_result_path =  BASE_DIR / 'nbs' / 'out' / 'work-dir' / 'provenance' / 'd2_1v4_ace_00006.json'
sample_result = EliminationSegmentationResult.from_json(sample_result_path.read_text())
sample_result

EliminationSegmentationResult(elimination_events=[EliminationEvent(event=SegmentationResult(bbox=XYXYBBox(dims=[320.96661376953125, 36.627685546875, 584.9686889648438, 54.144866943359375]), ocr=None), eliminator=SegmentationResult(bbox=XYXYBBox(dims=[320.96661376953125, 40.964388847351074, 434.2342987060547, 52.10789680480957]), ocr=OCRResult(text='RANOUAPEEK Hiccup', confidence=0.9002308249473572)), weapon=SegmentationResult(bbox=XYXYBBox(dims=[434.2342987060547, 40.964388847351074, 475.1927185058594, 52.10789680480957]), ocr=None), eliminated=SegmentationResult(bbox=XYXYBBox(dims=[475.1927185058594, 40.964388847351074, 584.9686889648438, 52.10789680480957]), ocr=OCRResult(text='Hutch wala kutta', confidence=0.9410244226455688))), EliminationEvent(event=SegmentationResult(bbox=XYXYBBox(dims=[320.46221923828125, 37.169036865234375, 501.333251953125, 53.19525146484375]), ocr=None), eliminator=SegmentationResult(bbox=XYXYBBox(dims=[320.46221923828125, 40.94595241546631, 434.4218368530273

In [57]:
print('Original events: ', len(sample_result.elimination_events))
deduped_result = remove_duplicate_events(sample_result)
print('Deduped events: ', len(deduped_result.elimination_events))

Original events:  2
Deduped events:  1
